In [ ]:
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import QuantileTransformer

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv('ravdess_features.csv')

In [ ]:
df_copy = df.copy()

# Data Semantics
Introduce the variables with their meaning and characteristics

ravdess_features.csv è un file csv contenente un dataset originato da dei file audio in cui sono registrate delle brevi frasi pronunciate da 24 attori, 12 maschi e 12 femmine, e l'emozione con cui sono dette. Ogni record nel dataset rappresenta uno di questi audio, e ha come feature delle caratteristiche dell'audio.

Features:  
    **modality** (audio-only) -> *categorical*  
    **vocal_channel** (speech, song) -> *categorical*  
    **emotion** (neutral, calm, happy, sad, angry, fearful, disgust, surprised) -> *categorical*  
    **emotional_intensity** (normal, strong). NOTE: There is no strong intensity for the 'neutral' emotion -> *ordinal*  
    **statement** ("Kids are talking by the door", "Dogs are sitting by the door") *categorical*  
    **repetition** (1st repetition, 2nd repetition) -> *categorical*  
    **actor** (01 to 24) -> *categorical*  
    **sex** (M, F) -> *categorical*  
    **channels** (number of channels; 1 for mono, 2 for stereo audio) -> *categorical*  
    **sample_width** (number of bytes per sample; 1 means 8-bit, 2 means 16-bit) -> *categorical*  
    **frame_rate** (frequency of samples used (in Hertz)) -> *categorical*  
    **frame_width** (Number of bytes for each frame. One frame contains a sample for each channel.) -> *categorical*  
    **length_ms** (audio file length (in milliseconds)) -> *numeric*  
    **frame_count** (the number of frames from the sample) -> *numeric*  
    **intensity** (loudness in dBFS (dB relative to the maximum possible loudness)) -> *numeric*  
    **zero_crossings_sum** (sum of the zero-crossing rate, where The zero-crossing rate (ZCR) is the rate at which a signal changes from positive to zero to negative or from negative to zero to positive in a single frame) -> *numeric*  
    **'mean', 'std', 'min', 'max', 'kur', 'skew'** (statistics of the original audio signal. kur è la curtosi, che rappresenta un allontanamento dalla normalità distributiva, rispetto alla quale si verifica un maggiore appiattimento o un maggiore allungamento. skewness is a measure of the asymmetry of the probability distribution of a real-valued random variable about its mean) -> *numeric*  
    **mfcc_ 'mean', 'std', 'min', 'max'** (statistics of the Mel-Frequency Cepstral Coefficients) -> *numeric*  
    ***Mel-Frequency Cepstral Coefficients*** (MFCCs) represent the spectral characteristics of an audio signal, specifically focusing on how humans perceive sound. They capture information about the frequency content of the signal, emphasizing frequencies that are more perceptually relevant (according to the Mel scale), and they represent this information in a compact form suitable for tasks like speech recognition, speaker identification, and audio classification. Essentially, MFCCs serve as a feature vector that summarizes the unique aspects of the audio signal's spectrum, making it easier for machine learning algorithms to process and analyze audio data.  
    **sc_ 'mean', 'std', 'min', 'max', 'kur', 'skew'** (statistics of the spectral centroid) -> *numeric*  
    ***The spectral centroid*** is a measure used in digital signal processing to characterise a spectrum. It indicates where the center of mass of the spectrum is located. Perceptually, it has a robust connection with the impression of brightness of a sound   
    **stft_ 'mean', 'std', 'min', 'max', 'kur', 'skew'** (statistics of the stft chromagram) -> *numeric*  
    ***STFT chromagram*** is a time-varying representation that provides insight into the harmonic content and chord progression of an audio signal representing the normalized distribution of energy in different pitch classes (or musical notes) over time

In [ ]:
continuous_col = ['length_ms', 'frame_count', 'intensity', 'zero_crossings_sum', 'mean', 'std', 'min', 'max', 'kur', 
                      'skew', 'mfcc_mean', 'mfcc_std', 'mfcc_min', 'mfcc_max', 'sc_mean', 'sc_std', 'sc_min', 'sc_max', 
                      'sc_kur', 'sc_skew', 'stft_mean', 'stft_std', 'stft_min', 'stft_max', 'stft_kur', 'stft_skew']
discrete_col = ['modality', 'vocal_channel', 'emotion', 'emotional_intensity', 'statement', 'repetition', 'actor', 
                    'sex', 'channels', 'sample_width', 'frame_rate', 'frame_width']

In [ ]:
df.columns

In [ ]:
df.info()

Notiamo subito che le variabili vocal_channel, intensity e actor contengono missing values.

In [ ]:
df.nunique()

Numero di valori distinti per variabile

Le variabili modality, sample_width, frame_rate e stft_max si potranno eliminare perché sono costanti.

In [ ]:
df.head()

# Distribution of the variables and statistics
Explore (single, pairs of…) variables quantitatively (e.g., statistics, distributions)

In [ ]:
pd.set_option('display.precision',2)
df.describe(include="all")

In [ ]:
fig = plt.figure(figsize=(15, 25)) 
fig_dims = (4, 3)

cols = discrete_col
i = 0
for col in cols:
    plt.subplot2grid(fig_dims, (i%4, i//4))
    df[col].value_counts().plot(kind='bar')
    plt.xticks(rotation=45)
    plt.xlabel(col.replace('_', ' '))
    i = i+1

plt.show()

In [ ]:
all(df[df['channels'] == 2].index == df[df['frame_width'] == 4].index)

quindi frame width = 2*channels

In [ ]:
df['channels'].value_counts()

Notiamo che le variabili channels e frame width sono estremamente sbilanciate. Essendo quasi costanti le potremo eliminare.
Decidiamo di eliminare anche la variabile actor perché oltre ad essere per quasi la metà missing, non contiene informazioni utili per la nostra analisi, che avrà come obbiettivo della parte di classificazione la predizione della variabile emotion.

In [ ]:
discrete_col_to_delete = ['modality', 'channels', 'sample_width', 'actor', 'frame_rate', 'frame_width']

In [ ]:
discrete_col_to_keep = [col for col in discrete_col if col not in discrete_col_to_delete]

In [ ]:
df['emotion'].value_counts().plot(kind='bar')
plt.show()

Le frequenze minori di surprised, disgust e neutral si spiegano perché i primi due sono valori ammissibili dell'attributo emotion quando il valore di vocal channel è speech, l'ultimo quando il valore di emotional intensity è normal.

In [ ]:
df['actor'].value_counts().plot(kind='bar')
plt.show()

Questo grafico in realtà non è molto significativo perché solo approssimativamente la metà dei valori di actor è non nullo.

In [ ]:
df['intensity'].iloc[0:200].plot()
plt.show()

In questo grafico si vede come l'attributo intensity abbia molti missing values.

In [ ]:
df['intensity'].plot()
plt.show()

In [ ]:
df['length_ms'].plot()
plt.show()

In [ ]:
df['frame_count'].plot()
plt.show()

In [ ]:
df['frame_count'].iloc[1400:1600].plot()
plt.show()

Da questi grafici si vedono possibili valori anomali dove frame count vale 0.
Notiamo tuttavia che frame_count dovrebbe essere legato a length_ms e frame_rate dalla relazione length_ms = frame_count/frame_rate*1000.

In [ ]:
length_diff = df['length_ms']-df['frame_count']/df['frame_rate']*1000
length_diff.plot()
plt.show()

Verifico che queste deviazioni corrispondano a quelle in frame_count

In [ ]:
((length_diff > 1) == (df['frame_count'] < 1)).all()

Dunque potremo considerare al posto delle tre variabili length_ms, frame_count, frame_rate, solo la variabile length_ms (in questo modo risolviamo anche il problema dei valori anomali di frame_count).

In [ ]:
continuous_col_to_delete = ['stft_max', 'frame_count']
continuous_col_to_keep = [col for col in continuous_col if col not in continuous_col_to_delete]

In [ ]:
df['zero_crossings_sum'].plot()
plt.show()

In [ ]:
df['zero_crossings_sum'].hist(bins=20)
plt.show()

In [ ]:
df[['mean','min','max','std','kur','skew']].hist(figsize=(10, 8), bins=20)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 12)) 
fig_dims = (3, 2)

cols = ['mean','min','max','std','kur','skew']
i = 0
for col in cols:
    if i <= 2:
        plt.subplot2grid(fig_dims, (i, 0))
    else:
        plt.subplot2grid(fig_dims, (i-3, 1))
    df[col].plot()
    #plt.xticks(rotation=45)
    plt.title(col.replace('_', ' '))
    i = i+1

plt.show()
plt.show()

Da questi grafici e istogrammi osserviamo che la variabile mean è schiacciata sullo zero e con molti outliers. 

In [ ]:
df['mean'][1500:1600].plot()
plt.title('mean')
plt.show()

In [ ]:
df.boxplot(column='mean')
plt.show()

Il boxplot conferma le nostre osservazioni sul grafico e l'istogramma di mean.

In [ ]:
df['kur'].plot()
plt.show()

In [ ]:
df.boxplot(column='kur')
plt.show()

Osserviamo che kur presenta molti outliers, ma non sembrano essere dovuti a errori nella raccolata dei dati e sembrano coerenti con i valori che assume la variabile.

In [ ]:
df[['mfcc_mean','mfcc_min','mfcc_max','mfcc_std']].hist(figsize=(10, 8), bins=20)
plt.show()

In [ ]:
fig = plt.figure(figsize=(14, 12)) 
fig_dims = (2, 2)

cols = ['mfcc_mean','mfcc_min','mfcc_max','mfcc_std']
i = 0
for col in cols:
    if i <= 1:
        plt.subplot2grid(fig_dims, (i, 0))
    else:
        plt.subplot2grid(fig_dims, (i-2, 1))
    df[col].plot()
    #plt.xticks(rotation=45)
    plt.title(col.replace('_', ' '))
    i = i+1

plt.show()
plt.show()

Le variabili mfcc hanno distribuzioni vicine a quella normale e non sembrano presentare particolari problemi.

In [ ]:
df[['stft_mean','stft_min','stft_max','stft_std','stft_kur','stft_skew']].hist(figsize=(10, 8), bins=20)
plt.show()

In [ ]:
(df['stft_min'] == 0).sum()

Interpretiamo i valori 0 di stft_min come degli errori di misurazione e li tratteremo quindi come missing values.

In [ ]:
fig = plt.figure(figsize=(10, 12)) 
fig_dims = (3, 2)

cols = ['stft_mean','stft_min','stft_max','stft_std','stft_kur','stft_skew']
i = 0
for col in cols:
    if i <= 2:
        plt.subplot2grid(fig_dims, (i, 0))
    else:
        plt.subplot2grid(fig_dims, (i-3, 1))
    df[col].plot()
    #plt.xticks(rotation=45)
    plt.title(col.replace('_', ' '))
    i = i+1

plt.show()
plt.show()

Queste variabi sembrano evidenziare un pattern periodico.

In [ ]:
df[['sc_mean','sc_min','sc_max','sc_std','sc_kur','sc_skew']].hist(figsize=(10, 8), bins=20)
plt.show()

In [ ]:
(df['sc_min'] == 0).sum()

Interpretiamo i valori 0 di sc_min come degli errori di misurazione e li tratteremo quindi come missing values.

In [ ]:
fig = plt.figure(figsize=(10, 12)) 
fig_dims = (3, 2)

cols = ['sc_mean','sc_min','sc_max','sc_std','sc_kur','sc_skew']
i = 0
for col in cols:
    if i <= 2:
        plt.subplot2grid(fig_dims, (i, 0))
    else:
        plt.subplot2grid(fig_dims, (i-3, 1))
    df[col].plot()
    #plt.xticks(rotation=45)
    plt.title(col.replace('_', ' '))
    i = i+1

plt.show()
plt.show()

In [ ]:
# in cols ho escluso intensity perchè la funzione boxplot non vuole missing values

fig = plt.figure(figsize=(12, 78)) 
fig_dims = (13, 2)
cols = [col for col in continuous_col if col != 'intensity']
i = 0
for col in cols:
    if i <= 12:
        plt.subplot2grid(fig_dims, (i, 0))
    else:
        plt.subplot2grid(fig_dims, (i-13, 1))
    df.boxplot(column=col)
    #plt.xticks(rotation=45)
    i = i+1

plt.show()
plt.show()

In generale osserviamo che le variabili presentano molto outliers, ma che nella maggior parte dei casi questi sembrano essere coerenti con i valori che la variabile assume.
Nel boxplot di sc_min i valori anomali 0 non vengono visualizzati come outliers perché sono più del 10% e quindi il whisker ha lunghezza zero.
Infine come ci aspettavamo nel boxplot di framecount vediamo degli outliers in corrispondenza dello 0.

# Scatter

In [ ]:
variables = [(df['mfcc_std'], df['std']), 
             (df['sc_std'], df['mfcc_mean']), 
             (df['stft_skew'], df['skew']), 
             (df['std'], df['max']),
             (df['mfcc_std'], df['min']),
             (df['mfcc_min'], df['std']),
             (df['mfcc_mean'], df['mfcc_min'])]
 
for var1, var2 in variables:
    plt.figure()
    sns.scatterplot(x=var1, y=var2)
    plt.xlabel(var1.name)
    plt.ylabel(var2.name)
    plt.title(f"Scatter Plot of {var1.name} vs {var2.name}")
    plt.show()

In questi grafici abbiamo raccolto le coppie di variabili che mostrano i vari tipi di relazione che si osservano nella scatter plot matrix.
In particolare nel caso di stft_skew e skew dallo scatter plot non emerge nessuna correlazione, mentre con mfcc_std e min sembra esserci una correlazione logaritmica.

In [ ]:
plt.figure()
sns.scatterplot(x=df_copy['mfcc_min'], y=np.log(df['std']))
plt.show()

# Correlazione

In [ ]:
#continuous_col_no_nan_const = [col for col in continuous_col if col not in ('intensity', 'stft_max')]

In [ ]:
correlation_matrix = df[continuous_col].drop(['stft_max'], axis=1).corr()
plt.figure(figsize = (10,10))
sns.heatmap(correlation_matrix, square=True)
plt.show()

In [ ]:
#correlation_series = pd.Series({(row, col): correlation_matrix.loc[row, col] for row in correlation_matrix.index for col in correlation_matrix.columns})
correlation_series = pd.Series({(correlation_matrix.index[i], correlation_matrix.columns[j]): 
                                correlation_matrix.loc[correlation_matrix.index[i], correlation_matrix.columns[j]] 
                                for i in range(len(correlation_matrix)) for j in range(1+i,len(correlation_matrix))})

Serie Pandas che contiene la correlazione di ogni coppia di variabili continue.

In [ ]:
max_correlation_series = pd.Series({(row, abs(correlation_matrix[row][correlation_matrix[row]<1]).idxmax()): 
                                    abs(correlation_matrix.loc[row, abs(correlation_matrix[row][correlation_matrix[row]<1]).idxmax()]) 
                                    for row in correlation_matrix.index})

Serie Pandas con solo le coppie più correlate.

In [ ]:
correlation_series.sort_values(axis=0, ascending=False, inplace=True, key=abs)

In [ ]:
correlation_series[correlation_series.abs() > 0.85]

Vista l'elevata correlazione andremo a eliminare le variabili intensity (in questo modo rimuoviamo anche il problema dei missing values di intensity), mfcc_min, min, max, stft_mean

In [ ]:
max_correlation_series.sort_values(axis=0)

Osserviamo che mean ha una correlazione bassissima con tutte le altre variabili.

In [ ]:
continuous_col_to_delete = ['frame_count', 'mean', 'stft_max', 'min', 'max', 'stft_mean', 'mfcc_min', 'intensity']
continuous_col_to_keep = [col for col in continuous_col if col not in continuous_col_to_delete]

In [ ]:
df_reduced = df[continuous_col_to_keep]
df_reduced['mean'] = df['mean']
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df_reduced)
df_reduced_scaled = pd.DataFrame(scaled_data, columns=df_reduced.columns)

In [ ]:
df_reduced_scaled.std()

Osserviamo che la varianza di mean è significativamente minore rispetto a quella delle altre variabili. In definitiva quindi decidiamo di eliminare la variabile mean.

## Missing Values

In [ ]:
null = df.isnull().sum()
perc = (null/(len(df)))*100
perc

Calcoliamo la percentuale dei valori mancanti. Mentre "vocal_channel" può essere sostituito, la colonna "actor" è difficilmente servibile a causa dell'altissimo numero di valori nulli. 

# Missing Values e Outliers

In [ ]:
continuous_col_to_delete = ['frame_count', 'mean', 'stft_max', 'min', 'max', 'stft_mean', 'mfcc_min', 'intensity']
continuous_col_to_keep = [col for col in continuous_col if col not in continuous_col_to_delete]
discrete_col_to_delete = ['modality', 'channels', 'sample_width', 'actor', 'frame_rate', 'frame_width']
discrete_col_to_keep = [col for col in discrete_col if col not in discrete_col_to_delete]

Rimangono da sistemare gli outliers di sc_min e stft_min (che trattiamo come valori errati e quindi missing values) e i missing values di vocal channel.
In praticolare per sc_min e stft_min sfruttiamo la regressione lineare (usando le variabili più correlate rispettivamente con sc_min e stft_min).

Linear Regression su sc_min

In [ ]:
df_copy.loc[df_copy['sc_min'] != 0, 'sc_min'].hist()

In [ ]:
df.loc[df['sc_min'] == 0, 'sc_min'] = np.nan

In [ ]:
df.loc[df['stft_min'] == 0, 'stft_min'] = np.nan

In [ ]:
df[continuous_col].corr()['sc_min'].sort_values(axis=0, key=abs, ascending=False)

In [ ]:
null_index = np.argwhere(df['sc_min'].isna().values).ravel()
not_nullindex = np.argwhere(~df['sc_min'].isna().values).ravel()

In [ ]:
X = df[['sc_mean', 'mfcc_max', 'zero_crossings_sum', 'mfcc_mean', 'sc_skew', 'sc_max']].values
X_train = X[not_nullindex]
y_train = df['sc_min'].values[not_nullindex]
X_test = X[null_index]

In [ ]:
reg = LinearRegression()
reg.fit(X_train,y_train)

In [ ]:
sc_min_pred = reg.predict(X_test)

In [ ]:
df.loc[null_index, 'sc_min'] = sc_min_pred

In [ ]:
df['sc_min'].hist()

la distribuzione è rimasta abbastanza simile a quella originaria e quindi consideriamo il risultato soddisfacente.

Linear Regression su stft_min

In [ ]:
df_copy.loc[df_copy['stft_min'] != 0, 'stft_min'].hist()

In [ ]:
df[continuous_col].corr()['stft_min'].sort_values(axis=0, key=abs, ascending=False)

In [ ]:
null_index2 = np.argwhere(df['stft_min'].isna().values).ravel()
not_nullindex2 = np.argwhere(~df['stft_min'].isna().values).ravel()

In [ ]:
X2 = df[['stft_std', 'stft_mean','stft_skew']].values
X_train2 = X2[not_nullindex2]
y_train2 = df['stft_min'].values[not_nullindex2]
X_test2 = X2[null_index2]

In [ ]:
reg2 = LinearRegression()
reg2.fit(X_train2,y_train2)

In [ ]:
stft_min_pred = reg2.predict(X_test2)

In [ ]:
df.loc[null_index2, 'stft_min'] = stft_min_pred

In [ ]:
df['stft_min'].hist()

La regressione ha aggiunto qualche valore negativo, ma la distribuzione è rimasta abbastanza simile a quella originaria e quindi consideriamo il risultato soddisfacente.

In ultima istanza si sostituiscono i valori mancanti di vocal_channel con la loro moda. 

In [ ]:
df_copy['vocal_channel'].value_counts()/len(df_copy)

In [ ]:
df['vocal_channel'] = df['vocal_channel'].fillna(df['vocal_channel'].mode()[0])

In [ ]:
df['vocal_channel'].value_counts()/len(df)

# Trasformazioni delle variabili

In [ ]:
qt = QuantileTransformer(output_distribution='normal')

Quantile tranformer sfrutta la funzione di ripartizione (approssimata) di una variabile per trasformare la variabile in modo che la sua distribuzione diventi normale.

In [ ]:
df_norm = df[continuous_col_to_keep]

In [ ]:
for column in df_norm.columns:
    df_norm.loc[:, column] = qt.fit_transform(df[[column]])

In [ ]:
# Visualize distribution of multiple columns
plt.figure(figsize=(30, 40), dpi=500)
num_columns = len(df_norm.columns)

for i, column in enumerate(df_norm.columns):
    plt.subplot(num_columns, 2, i + 1)
    sns.histplot(df[column], kde=True)
    plt.title(f'Before Transformation - {column}')

    plt.subplot(num_columns, 2, i + num_columns + 1)
    sns.histplot(df_norm[column], kde=True)
    plt.title(f'After Transformation - {column}')

plt.tight_layout()
plt.show()

## Commento finale

Siamo riusciti a preparare i dati per i successivi lavori di clustering e classificazione. Abbiamo creato un file csv appositamente modificato in modo tale da dover apporre solo data preparation apposite per la tipologia di algoritmo che andremo ad utilizzare. 